<a href="https://colab.research.google.com/github/zhangling297/Substance-Use/blob/master/Quick_EDA_demenstration__2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
title: "County Health Rankings – Quick EDA"
author: "Ling Zhang"
date: "`r Sys.Date()`"
output: pdf_document

In [ ]:
1) Setup & data import
# --- install once in your environment (uncomment to run) ---
# !pip install pandas numpy matplotlib tableone

import pandas as pd
import numpy as np
from pathlib import Path

# File paths – adjust if needed
csv_path = Path("Desktop/2023 County Health Rankings Data - v2 (1).csv")

# Read the CSV with original column names preserved
counties = pd.read_csv(csv_path)
print(counties.shape)
counties.head()

2) Helper: find probable column names & coerce numeric
def pick_first_present(df, candidates):
    """Return the first column name from candidates that exists in df; else None."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

# Candidate names based on your R code (add/adjust if your file differs)
food_candidates   = ["Food.Insecurity.raw.value", "food_insecurity_raw_value", "Food_Insecurity.raw.value"]
obesity_candidates= ["Obesity.raw.value", "obesity_raw_value", "Obesity.raw.value "]
severe_cost_candidates = ["Severe.Housing.Cost.Burden.raw.value", "severe_housing_cost_burden_raw_value"]

food_col   = pick_first_present(counties, food_candidates)
obese_col  = pick_first_present(counties, obesity_candidates)
severe_col = pick_first_present(counties, severe_cost_candidates)

if food_col is None or obese_col is None:
    raise ValueError("Could not find Food Insecurity and/or Obesity columns. Check your CSV headers.")

# Safely coerce to numeric
counties["food_insecurity"] = pd.to_numeric(counties[food_col], errors="coerce")
counties["obesity"]         = pd.to_numeric(counties[obese_col], errors="coerce")

# Optional: create A/B like in your R code (edit as appropriate)
# A <- "Severe.Housing.Cost.Burden.raw.value"; B <- "Percentage" (if it exists)
if severe_col is not None:
    counties["A"] = pd.to_numeric(counties[severe_col], errors="coerce")
else:
    counties["A"] = np.nan  # or drop if not available

# If you truly have a "Percentage" column:
percent_candidates = ["Percentage", "percentage"]
perc_col = pick_first_present(counties, percent_candidates)
if perc_col is not None:
    counties["B"] = pd.to_numeric(counties[perc_col], errors="coerce")
else:
    counties["B"] = np.nan  # placeholder if not present

counties[["food_insecurity", "obesity", "A", "B"]].describe()

In [ ]:
3) Quartiles & two-group split (like your R cut and food_ins_two_cat)
# Quartiles for food insecurity
counties["food_insecurity_quartile"] = pd.qcut(
    counties["food_insecurity"],
    q=4,
    labels=["Q1 (low)", "Q2", "Q3", "Q4 (high)"]
)

# Two-group split at the median
fi_median = counties["food_insecurity"].median(skipna=True)
counties["food_ins_two_cat"] = np.where(
    counties["food_insecurity"] <= fi_median, "Low", "High"
)
counties["food_ins_two_cat"] = pd.Categorical(counties["food_ins_two_cat"], categories=["Low", "High"])
counties[["food_insecurity", "food_insecurity_quartile", "food_ins_two_cat"]].head()

import matplotlib.pyplot as plt

# Scatter plot
ax = counties.plot.scatter(x="food_insecurity", y="obesity", alpha=0.5)
ax.set_xlabel("Proportion of Households with Food Insecurity")
ax.set_ylabel("Proportion of Adults with Obesity")
plt.show()

# Histogram / density approximations can be added similarly
5) TableOne-style summary (overall and stratified)
A) Using the tableone package (closest to R’s CreateTableOne)
from tableone import TableOne

# Choose variables to summarize (continuous + categorical)
# Edit these to match your columns of interest
variables   = ["food_insecurity", "obesity", "A", "B", "food_insecurity_quartile"]
categorical = ["food_insecurity_quartile"]  # add more if you have other categoricals
# Note: A/B may be continuous; move them to categorical if they truly are categories.

# Overall TableOne
tableone_overall = TableOne(
    data=counties,
    columns=variables,
    categorical=categorical,
    missing=True
)
print(tableone_overall.tabulate(tablefmt="github"))

# Stratified by the two-group split
tableone_strat = TableOne(
    data=counties,
    columns=variables,
    categorical=categorical,
    groupby="food_ins_two_cat",
    missing=True
)
print(tableone_strat.tabulate(tablefmt="github"))


###variable labels or non-default statistics, see TableOne docs (you can pass labels, nonnormal, pval, etc.).

B) Pure-pandas fallback (if you prefer not to install tableone)
# Define which are continuous vs categorical
cont_vars = ["food_insecurity", "obesity", "A", "B"]
cat_vars  = ["food_insecurity_quartile"]

# Overall summary (continuous)
cont_summary = counties[cont_vars].agg(["count", "mean", "std", "min", "median", "max"]).T

# Overall summary (categorical)
cat_summary = (
    counties[cat_vars].apply(lambda s: s.value_counts(dropna=False))
    .fillna(0)
)

print("== Overall: Continuous ==")
display(cont_summary)
print("\n== Overall: Categorical ==")
display(cat_summary)

# Stratified by food_ins_two_cat
group = counties.groupby("food_ins_two_cat", dropna=True)

# Continuous by group
cont_by_group = group[cont_vars].agg(["count", "mean", "std", "min", "median", "max"])
print("\n== Stratified Continuous (by food_ins_two_cat) ==")
display(cont_by_group)

# Categorical by group: frequency tables
print("\n== Stratified Categorical (by food_ins_two_cat) ==")
for v in cat_vars:
    ctab = pd.crosstab(counties[v], counties["food_ins_two_cat"], margins=True, dropna=False)
    print(f"\n{v}:\n", ctab)

6) Quick checks akin to your R summary() and numeric coercion tests
# Class / dtype
print(counties["food_insecurity"].dtype)

# NA count
count_na = counties["food_insecurity"].isna().sum()
print("NAs in food_insecurity:", count_na)

# Summary
print(counties["food_insecurity"].describe())

7) Optional: simple statistical tests (t-test & chi-square analogs)
from scipy import stats

# t-test: obesity by Low vs High food insecurity
tt = counties.dropna(subset=["obesity", "food_ins_two_cat"])
grp_low  = tt.loc[tt["food_ins_two_cat"]=="Low",  "obesity"]
grp_high = tt.loc[tt["food_ins_two_cat"]=="High", "obesity"]
t_stat, p_val = stats.ttest_ind(grp_low, grp_high, equal_var=False, nan_policy="omit")
print(f"t-test obesity ~ food_ins_two_cat: t={t_stat:.3f}, p={p_val:.3g}")

# Chi-square: quartile vs two-cat
chi_df = counties.dropna(subset=["food_insecurity_quartile", "food_ins_two_cat"])
chi_table = pd.crosstab(chi_df["food_insecurity_quartile"], chi_df["food_ins_two_cat"])
chi2, p, dof, exp = stats.chi2_contingency(chi_table)
print("\nChi-square test (quartile vs two-cat):")
print("chi2=", chi2, "p=", p, "dof=", dof)
